In [1]:

import math
from copy import deepcopy
import sys
def decompress_gz(file_path, output_folder):
    output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(file_path))[0])
    with gzip.open(file_path, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    return output_file

def dist(a,b):
    return math.sqrt(sum([(a[i]-b[i])**2 for i in range(min(len(a),len(b)))]))

def angle(a,b,c):
    """Calculate the angle between three points. First point in the middle"""
    d12 = dist(a, b)
    d13 = dist(a, c)
    d23 = dist(b, c) 
    #round. To avoid things like 1.000000001
    angle = math.acos(round((d12**2 + d13**2 - d23**2)/(2*d12*d13),7))
    return angle

def angled(a,b,c):
    return angle(a,b,c)*180/math.pi

def dihedral(a,b,c,d):
    """ Calculate dihedral considering a in the beggining"""
    v1 = [b[i] - a[i] for i in range(3)]
    v2 = [c[i] - b[i] for i in range(3)]
    v3 = [d[i] - c[i] for i in range(3)]
    temp = [dist((0,0,0),v2) * v1[i] for i in range(3)]
    y = dotProd(temp ,crossProd(v2,v3))
    x = dotProd(crossProd(v1,v2),crossProd(v2,v3))
    rad = math.atan2(y,x)
    return rad*(180/math.pi) 

def dotProd(a,b):
    N = min(len(a),len(b))
    return sum([a[i] * b[i] for i in range(N)])

def crossProd(a,b):
    """Pretty self-explanatory, this function bakes cookies"""
    normal_vect = [
    a[1]*b[2] - a[2]*b[1],
    a[2]*b[0] - a[0]*b[2],
    a[0]*b[1] - a[1]*b[0]]
    return normal_vect

def rawVec(a,b):
    N = min(len(a),len(b))
    return [b[i]-a[i] for i in range(N)]

class PDBQT():
    def __init__(self, line):
        self._parse_common(line)     # used here (PDB) and in PDBQT
        self._parse_specific(line)   # differs in PDBQT

    def getline(self):
        txt = self._print_common()    # no \n; PDB + PDBQT
        txt += self._print_specific() # differs in PDBQT
        return txt

    def dist(self, a):
        return dist(self.getcoords(), a.getcoords())

    def angle(self, a, b):
        return angled(self.getcoords(), a.getcoords(), b.getcoords())

    def getcoords(self):
        return (self.x, self.y, self.z)

    def setcoords(self, coords):
        self.x, self.y, self.z = coords
    # blanks: [11:12], [20:21], [27:30], [66:76](pdb) or [66:68](pdbqt)

    def _parse_common(self, line):
        """Common to PDB and PDBQT formats"""
        self.keyword     = line      [ 0: 6]     # ATOM or HETATM
        self.serial      = int(line  [ 6:11])    # atom id
        #                            [11:12]
        self.name        = line      [12:16]     # atom name
        self.altLoc      = line      [16:17]     # Alternate location
        self.resName     = line      [17:20]     # Residue name
        #                            [20:21] 
        self.chain       = line      [21:22]     # chain
        self.resNum      = int(line  [22:26])    # Residue number
        self.icode       = line      [26:27]     # ???
        #                            [27:30]
        self.x           = float(line[30:38])    # X
        self.y           = float(line[38:46])    # Y
        self.z           = float(line[46:54])    # Z
        self.occupancy   = float(line[54:60])    # Occupancy
        self.bfact       = float(line[60:66])    # Temperature factor

    def _parse_specific(self, line):
        """ PDBQT characters [68:79] """
        #self.charge      = float(line[68:76])   # Charge
        self.atype       = line      [77:79]    # Atom type
        self.atype = self.atype.strip().upper()
        self.atomnr = self.atype_to_atomnr(self.atype)

    def _print_common(self):
        """ Characters [0:68]"""
        linestr = ''
        linestr += '%6s' % (self.keyword)
        linestr += '%5d' % (self.serial)
        linestr += ' ' 
        linestr += '%4s' % (self.name)
        linestr += '%1s' % (self.altLoc) 
        linestr += '%3s' % (self.resName)
        linestr += ' ' 
        linestr += '%1s' % (self.chain)
        linestr += '%4d' % (self.resNum)
        linestr += '%1s' % (self.icode)
        linestr += ' ' * 3 
        linestr += '%8.3f' % (self.x)
        linestr += '%8.3f' % (self.y)
        linestr += '%8.3f' % (self.z)
        linestr += '%6.2f' % (self.occupancy)
        linestr += '%6.2f' % (self.bfact)
        return linestr

    def _print_specific(self):
        """ PDBQT characters [68:79] """
        linestr =  ' ' * 2                      # [66:68]
        linestr += '%8.3f' % (self.charge)      # [68:76]
        linestr += ' ' * 1                      # [76:77]
        linestr += '%2s' % (self.atype)       # [77:79]
        linestr += '\n'
        return linestr


    def dist(self, other):
        s = [(a-b)**2 for (a, b) in zip(self.getcoords(), other.getcoords())]
        return math.sqrt(sum(s))

    def isbound(self, other_atom, cut_off_percent = .5):
        """ Depends on atomic number """
        threshold = cut_off_percent * (
            .5 * self.atomnr_vdw(self.atomnr) +
            .5 * self.atomnr_vdw(other_atom.atomnr))
        is_bound = self.dist(other_atom) < threshold
        return is_bound

    def atomnr_vdw(self, atomnr):
        D = {1:2.0, 6:4.0, 7:3.5, 8:3.2, 9:3.1, 12:1.3, 15:4.2, 16:4.0, 17:4.1, 
        20:2.0, 25:1.3, 26:1.3, 30:1.5, 35:4.3, 53:4.7} 
        return D[atomnr]

    def atype_to_atomnr(self, atype):
        D = {'H':1, 'HD':1, 'HS':1, 'C':6, 'A':6, 'N':7, 'NA':7, 'NS':7,'O':8, 
             'OA':8,'OS':8, 'F':9, 'MG':12, 'S':16, 'SA':16, 'CL':17, 
             'CA':20, 'MN':25, 'FE':26, 'ZN':30, 'BR':35, 'I':53, 'G':6, 
             'J':6, 'P':15, 'Z':6, 'GA':6, 'Q':6, 'TZ':-1}
        try:
            return D[atype.strip().upper()]
        except:
    #         sys.stderr.write(
    # 'unexpected atom type: %s (not in standard forcefield)\n' % atype)
            return -1 

def load_pdbqt(filename):
    """Creates a list of PDBQT atom objects"""
    atoms_list = []
    max_id = 0
    num_tz = 0
    non_atom_text = {}
    f = open(filename)
    counter = 0
    for line in f:
        if line.startswith('ATOM  ') or line.startswith('HETATM'):
            atom = PDBQT(line)
            if atom.atype == 'TZ':
                num_tz += 1
            else:
                counter += 1
                if atom.atype.upper() == 'CA':
                    # set zinc charge to zero
                    atom.charge = 0.0
                atoms_list.append(atom)
                max_id = max(max_id, atom.serial)
        else:
            if counter not in non_atom_text:
                non_atom_text[counter] = []
            non_atom_text[counter].append(line)
    f.close()
    if counter in non_atom_text: # text after all atoms
        non_atom_text['last'] = non_atom_text.pop(counter)
    return atoms_list, num_tz, max_id, non_atom_text

def bruteNearbyAtoms(atomsList, atype = 'CA', cutOff = 2.5):
    """Find atoms close to given atype in pdb/pdbqt"""
    metalsList = []
    for a in atomsList:
        #print(a.atype)
        if a.name.replace(" ","") == atype and a.resName.replace(" ","") == atype:
            metalsList.append(a)
            
    allNearbyLists = []
    #print(PDBQT._print_common(metalsList[0]))
    for metal in metalsList:
        bht_indx = []
        bht_dist = []
        for (i, atom) in enumerate(atomsList):
            if atom.dist(metal) < cutOff and atom.resName.replace(" ","")!="HOH":
                bht_indx.append(i)
                bht_dist.append(atom.dist(metal))
        idx = [i[0] for i in sorted(enumerate(bht_dist),
                 key = lambda x:x[1])]
        nearbyAtoms = []
        for i in idx:
            nearbyAtoms.append(atomsList[bht_indx[i]])
        allNearbyLists.append(nearbyAtoms)
    return allNearbyLists


In [2]:
import os
import gzip
import shutil

# Folder containing the .gz files
folders = [f for f in os.listdir("./") if os.path.isdir(os.path.join("./", f))]


for folder in folders:
    for filename in os.listdir(folder):
        if filename.endswith('.gz'):
            file_path = os.path.join(folder, filename)
            decompress_gz(file_path, folder)
            print("Decompressed:", filename)


('Decompressed:', '1bfd.pdb.gz')
('Decompressed:', '1bn8.pdb.gz')
('Decompressed:', '1cb8.pdb.gz')
('Decompressed:', '1ee6.pdb.gz')
('Decompressed:', '1g5c.pdb.gz')
('Decompressed:', '1gtt.pdb.gz')
('Decompressed:', '1gxo.pdb.gz')
('Decompressed:', '1hm2.pdb.gz')
('Decompressed:', '1hm3.pdb.gz')
('Decompressed:', '1hmu.pdb.gz')
('Decompressed:', '1hmw.pdb.gz')
('Decompressed:', '1i7o.pdb.gz')
('Decompressed:', '1j0m.pdb.gz')
('Decompressed:', '1j0n.pdb.gz')
('Decompressed:', '1j1t.pdb.gz')
('Decompressed:', '1jg8.pdb.gz')
('Decompressed:', '1lw4.pdb.gz')
('Decompressed:', '1lw5.pdb.gz')
('Decompressed:', '1m6s.pdb.gz')
('Decompressed:', '1n39.pdb.gz')
('Decompressed:', '1n3a.pdb.gz')
('Decompressed:', '1nkg.pdb.gz')
('Decompressed:', '1o5k.pdb.gz')
('Decompressed:', '1o88.pdb.gz')
('Decompressed:', '1o8d.pdb.gz')
('Decompressed:', '1o8e.pdb.gz')
('Decompressed:', '1o8f.pdb.gz')
('Decompressed:', '1o8g.pdb.gz')
('Decompressed:', '1o8h.pdb.gz')
('Decompressed:', '1o8j.pdb.gz')
('Decompre

In [4]:
# atoms_list, num_tz, max_id, non_atom_text=load_pdbqt("4tln.pdb")

# atoms_by_ca=bruteNearbyAtoms(atoms_list)

    

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# List of three-letter codes for standard amino acids
amino_acid_codes = [
    'ALA',  # Alanine
    'ARG',  # Arginine
    'ASN',  # Asparagine
    'ASP',  # Aspartic Acid
    'CYS',  # Cysteine
    'GLU',  # Glutamic Acid
    'GLN',  # Glutamine
    'GLY',  # Glycine
    'HIS',  # Histidine
    'ILE',  # Isoleucine
    'LEU',  # Leucine
    'LYS',  # Lysine
    'MET',  # Methionine
    'PHE',  # Phenylalanine
    'PRO',  # Proline
    'SER',  # Serine
    'THR',  # Threonine
    'TRP',  # Tryptophan
    'TYR',  # Tyrosine
    'VAL'   # Valine
]

# Print the list of three-letter codes

def graph_ca_interaction(atoms_by_ca, file_name):
    plt.ion()
    
    # Adjust coordinates
    for i in range(len(atoms_by_ca)):
        ca_list=atoms_by_ca[i]
        if len(ca_list)<2:
            continue
        ca = ca_list[0]
        adjusted_coordinates = [(0, 0, 0)]
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        int_lig=0
        int_receptor=0
        for atom in ca_list[1:]:
            atom.dist(ca)
            adjusted_coord = (atom.x - ca.x,
                            atom.y - ca.y,
                            atom.z - ca.z)
            adjusted_coordinates.append(adjusted_coord)

            # Draw a line from ca to adjusted_coord
            if atom.resName.replace(" ","") not in amino_acid_codes:
                ax.plot([0, adjusted_coord[0]],
                        [0, adjusted_coord[1]],
                        [0, adjusted_coord[2]], color='b')
                int_lig+=1
            else:
                ax.plot([0, adjusted_coord[0]],
                        [0, adjusted_coord[1]],
                        [0, adjusted_coord[2]], color='g')
                int_receptor+=1
            
        print("Atom "+str(i)+" of file "+file_name+" has "+str(int_lig)+" interactions with ligand and "+str(int_receptor)+" interactions with receptor")
        
        # Extracting x, y, z coordinates
        x_coords = [coord[0] for coord in adjusted_coordinates]
        y_coords = [coord[1] for coord in adjusted_coordinates]
        z_coords = [coord[2] for coord in adjusted_coordinates]

        # Color list: blue for the first element, red for others
        colors = ['b'] + ['r'] * (len(adjusted_coordinates) - 1)
        sizes = [100] + [20] * (len(adjusted_coordinates) - 1)
        
        # Plot the points with specified colors
        sc = ax.scatter(x_coords, y_coords, z_coords, c=colors, s=sizes, marker='o')

        # Setting labels
        ax.set_xlabel('X axis')
        ax.set_ylabel('Y axis')
        ax.set_zlabel('Z axis')
        ax.set_xlim(-3, 3)
        ax.set_ylim(-3, 3)
        ax.set_zlim(-3, 3)

        # Set the title
        ax.set_title('3D Scatter Plot')

        # Show the plot
        plt.show()
        fig.savefig(file_name+str(i)+'.png', dpi=300)


    # Turn off interactive mode
    plt.ioff()



#graph_ca_interaction(atoms_by_ca,"4tln")

In [8]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import os
import glob

# List of three-letter codes for standard amino acids
amino_acid_codes = [
    'ALA',  # Alanine
    'ARG',  # Arginine
    'ASN',  # Asparagine
    'ASP',  # Aspartic Acid
    'CYS',  # Cysteine
    'GLU',  # Glutamic Acid
    'GLN',  # Glutamine
    'GLY',  # Glycine
    'HIS',  # Histidine
    'ILE',  # Isoleucine
    'LEU',  # Leucine
    'LYS',  # Lysine
    'MET',  # Methionine
    'PHE',  # Phenylalanine
    'PRO',  # Proline
    'SER',  # Serine
    'THR',  # Threonine
    'TRP',  # Tryptophan
    'TYR',  # Tyrosine
    'VAL'   # Valine
]

# Print the list of three-letter codes

def ca_interaction():    

    folders = [f for f in os.listdir("./") if os.path.isdir(os.path.join("./", f))]
    data = {
        'file_name': [],
        'atom_name': [],
        'atom_ResName': [],
        'atom_Serial': [],
        'ligand_interactions': [],
        'receptor_interactions': []
    }
    # Create the DataFrame
    df = pd.DataFrame(data)
    for folder in folders:
        print("processing",folder)
        pattern = os.path.join("./"+folder, "*.pdb")
        files = glob.glob(pattern)
        for file in files:
            atoms_list, num_tz, max_id, non_atom_text = load_pdbqt(file)
            atoms_by_ca=bruteNearbyAtoms(atoms_list)
            for i in range(len(atoms_by_ca)):
                ca_list=atoms_by_ca[i]
                if len(ca_list)<2:
                    continue
                ca = ca_list[0]
                ligand_interactions = []
                receptor_interactions=[]
                ligand_at_type=[]
                receptor_at_type=[]
                int_lig=0
                int_receptor=0
                for atom in ca_list[1:]:
                    atom.dist(ca)
                    adjusted_coord = (atom.x - ca.x,
                                    atom.y - ca.y,
                                    atom.z - ca.z)

                    # Draw a line from ca to adjusted_coord
                    if atom.resName.replace(" ","") not in amino_acid_codes:
                        int_lig+=1
                        ligand_interactions.append(adjusted_coord)
                        ligand_at_type.append(atom.name)
                    else:
                        int_receptor+=1
                        receptor_interactions.append(adjusted_coord)
                        receptor_at_type.append(atom.name)

                df = df.append({
                    'file_name': file,
                    'atom_name': ca.name,
                    'atom_ResName': ca.resName,
                    'atom_Serial': ca.resNum,
                    'ligand_interactions': ligand_interactions,
                    'num_lig_int': int_lig,
                    'ligand_atoms':ligand_at_type,
                    'receptor_interactions': receptor_interactions,
                    'num_receptor_int':int_receptor,
                    'receptor_atoms':receptor_at_type
                    }, ignore_index=True)
            print(file,"completed")
        
    return df

In [9]:
df=ca_interaction()
print(df)
csv_file_path = 'calcium_analysis.csv'
df.to_csv(csv_file_path, index=False)

('processing', 'batch-download-structures-1722270746034')
('./batch-download-structures-1722270746034\\1bfd.pdb', 'completed')
('./batch-download-structures-1722270746034\\1bn8.pdb', 'completed')
('./batch-download-structures-1722270746034\\1cb8.pdb', 'completed')
('./batch-download-structures-1722270746034\\1ee6.pdb', 'completed')
('./batch-download-structures-1722270746034\\1g5c.pdb', 'completed')
('./batch-download-structures-1722270746034\\1gtt.pdb', 'completed')
('./batch-download-structures-1722270746034\\1gxo.pdb', 'completed')
('./batch-download-structures-1722270746034\\1hm2.pdb', 'completed')
('./batch-download-structures-1722270746034\\1hm3.pdb', 'completed')
('./batch-download-structures-1722270746034\\1hmu.pdb', 'completed')
('./batch-download-structures-1722270746034\\1hmw.pdb', 'completed')
('./batch-download-structures-1722270746034\\1i7o.pdb', 'completed')
('./batch-download-structures-1722270746034\\1j0m.pdb', 'completed')
('./batch-download-structures-1722270746034\\

: 

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'calcium_analysis.csv'
df = pd.read_csv(csv_file_path)

# Group by ligand_interactions and receptor_interactions and count entries
interaction_counts = df.groupby(['num_lig_int', 'num_receptor_int']).size().reset_index(name='count')

# Display the result
print(interaction_counts)

# Save the result to a new CSV file
interaction_counts.to_csv('interaction_counts.csv', index=False)

print("Interaction counts saved to interaction_counts.csv")


    num_lig_int  num_receptor_int  count
0           0.0               1.0    126
1           0.0               2.0     67
2           0.0               3.0    103
3           0.0               4.0     84
4           0.0               5.0     49
5           0.0               6.0     39
6           0.0               7.0      5
7           1.0               0.0      8
8           1.0               1.0     16
9           1.0               2.0     18
10          1.0               3.0     39
11          1.0               4.0      4
12          1.0               5.0      4
13          2.0               0.0     13
14          2.0               1.0     14
15          2.0               2.0     23
16          2.0               3.0     80
17          2.0               4.0      6
18          2.0               5.0      8
19          3.0               0.0      2
20          3.0               1.0      3
21          3.0               2.0     26
22          3.0               3.0     15
23          4.0 